In [1]:
import pandas as pd
import json
from pathlib import Path
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

%cd evaluation/ustdy_2_preference

/Users/yasith/projects/streaminghub/evaluation/ustdy_2_preference


In [2]:
data_path = Path.cwd() / "data"

titles = dict(
    q1="Readability",
    q2="Maintainability",
    q3="Modular Design",
    q4="Reusability",
    q5="Extensibility",
    q6="Debugging",
    q7="Learning Curve",
    q8="Dependencies",
    q9="Error Handling",
    q10="Overall",
    q11="Refactoring A to C",
    q12="Refactoring B to C",
)

rank_qs = ["q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10"]
rtng_qs = ["q11", "q12"]

data = {}
for path in data_path.glob("*.json"):
    with open(path) as f:
        answer = json.load(f)
        for k, v in answer.items():
            answer[k] = int(v) if k in rtng_qs else v
        data[path.stem] = answer

df = pd.DataFrame(data)

In [3]:
from matplotlib import pyplot as plt

r = 12 / 16

fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 4 * r), sharex="all", sharey="all")
for i, q in enumerate(rank_qs):
    # earlier it appears, higher the points
    df_points = pd.DataFrame()
    df_points["A"] = 3 - df.loc[q].str.index("A") # type: ignore
    df_points["B"] = 3 - df.loc[q].str.index("B") # type: ignore
    df_points["C"] = 3 - df.loc[q].str.index("C") # type: ignore
    df_points = df_points.mean()
    j, k = i // 5, i % 5
    ax = axs[j][k]
    df_points.plot.bar(title=f"{q.upper()} - {titles[q]}", ax=ax)
    ax.set_ylim(0, 3)
plt.suptitle("Rank-Based Score (Higher is Better)")
plt.tight_layout()
plt.savefig("figures/q_pref.pdf")
plt.close()

qdata = df.loc[rtng_qs].T.mean()
qdata.index = pd.Index(["A", "B"])
qdata.plot.bar(title=f"Difficulty of Migration")
plt.ylim(1, 5)
plt.tight_layout()
plt.savefig("figures/q_rating.pdf")
plt.close()